**Table of contents**

>[Description](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=mfCvdfDjfCbK)

>[Import Libraries](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=Zrj6oh8IWsPz)

>[Import dataset](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=uvOTUnErYOxg)

>[Data Preprocessing](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=UwsZI4ywYyR7)

>[Create Deep Learning Model](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=2gnaMaU4apLl)

>>[Batching](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=N5og9VUdbBid)

>>[Setting Hyper Parameters, Loss Functions, and Optimizer](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=HvZO6C-lbHdm)

>>[Train the model](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=jtQ6hHwmbcgX)

>[Design User Interface](#updateTitle=true&folderId=11cmakAGPTH1PlyzXpA-EGRJVemg3GGFH&scrollTo=2Zf1QB5hcEft)



# Description

Robin is my "final project" as a bachelor of computer engineering. In this project, I first created a dataset in Persian. This dataset includes three main parts, TAG, PATTERNS, and RESPONSES. The approach is that I categorized some of the possible questions. For example, the input from the user can be 1-Hi, 2_Hello, 3_How are you, 4_How are you doing, etc. All of these questions belong to one category called GREETING. So their tag would be greeting. On the other hand, each tag provides some pre-specified answers(called RESPONSES in the dataset) hence after Robin understands the tag based on user input, it will refer to its database and choose one of the pre-specified answers randomly. In fact, Robin learns the tags based on user input.

# Import Libraries

**Note: If you want to run this code, you first need to install the Hazm(a library for processing Persian words).**

In [2]:
!pip install hazm

In [3]:
import torch
from torch import nn
import json
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
import tkinter as tk
from tkinter import scrolledtext
import hazm
import torch
import random
from PIL import Image, ImageTk

# Import dataset

In [6]:
with open(r'data.json path', encoding='utf-8', errors='ignore') as input_data:
    data = json.load(input_data)
data

{'data': [{'tag': 'greeting',
   'patterns': ['سلام',
    'خوبی',
    'درود بر تو',
    'حالت چطوره',
    'احوالت چطوره',
    'سلام بر تو',
    'سلام علیکم',
    'سلام روز بخیر'],
   'responses': ['سلام بر شما، در خدمتم.',
    'سلام بر شما، بفرمایید.',
    'سلام، وقت شما بخیر. بفرمایید.',
    'سلام، امیدوارم حال شما خوب باشه. بفرمایید.',
    'درود برای خدمت رسانی آماده ام',
    'درود دوست عزیزم، جانم']},
  {'tag': 'goodbye',
   'patterns': ['خداحافظ.',
    'بدرود.',
    'در پناه حق.',
    'به امید دیدار',
    'خدا نگه دار',
    'فعلا خدا نگهدار'],
   'responses': ['در پناه حق.',
    'مواظب خودت باش.',
    'به امید دیدار.',
    'مواظب باش',
    'خدا حافظ']},
  {'tag': 'funny',
   'patterns': ['یه جوک برام تعریف کن.',
    'یک جوک خنده دار بگو.',
    'یه لطیفه تعریف کن.',
    'یه چیز خنده دار بگو.',
    'میتونی یه جوک برام تعریف کنی',
    'چه چیزای خنده داری بلدی',
    'بهم لطیفه بگو',
    'بهم جوک بگو'],
   'responses': ['با این قیمت طلا دیگه نمیشه حلقه خرید. باید یه اسپری قرمز بگیریم رو

# Data Preprocessing

We have to preprocess the data and separate them into X_train and y_train

In [7]:
stemmer = hazm.stemmer.Stemmer()
stemmer

In [22]:
stop_words = ["؟", "!", ".", "،", ":", "؛", "?", "/", "`", "~", '', ' ']
tags = []
words = []
pattern_tags = []

In [23]:
for instance in data["data"]:
    tag = instance["tag"]
    tags.append(tag)

    for pattern in instance["patterns"]:
        tokenized_pattern = hazm.word_tokenize(pattern)
        stemmed_pattern = [stemmer.stem(word) for word in tokenized_pattern if word not in stop_words]
        words.extend(stemmed_pattern)
        pattern_tags.append((stemmed_pattern, tag))

In [25]:
words = sorted(set(words))
words[:10]

['', 'آفرین', 'آنفولانزا', 'ا', 'ابر', 'احوال', 'از', 'اس', 'استاد', 'اطلاع']

In [26]:
def encoding(source, words):
    src = [stemmer.stem(word) for word in source]
    one_hot_encoding = np.zeros(len(words), dtype='float32')

    for index, word in enumerate(words):
        if word in src:
            one_hot_encoding[index]=1

    return one_hot_encoding

In [27]:
X_train = []
y_train = []

for X, y in pattern_tags:
    X = encoding(X, words)
    X_train.append(X)

    y = tags.index(y)
    y_train.append(y)

X_train = torch.from_numpy(np.array(X_train)).to(dtype=torch.float)
y_train = torch.from_numpy(np.array(y_train)).to(dtype=torch.long)

# Create Deep Learning Model

The deep learning model has 4 layers:

*First Layer*: input: len(words) units,      output: 120 units

*Second Layer*: input: 120 units,            output: 64 units

*Third Layer*: input: 64 units,              output: 36 units

*Fourth Layer*: input: 36 units,             output: len(tags) units

In [28]:
class ChatBot(nn.Module):
    def __init__(self, inp_size, out_size):
        super().__init__()
        self.layer1 = nn.Linear(inp_size,120)
        self.layer2 = nn.Linear(120,64)
        self.layer3 = nn.Linear(64,36)
        self.layer4 = nn.Linear(36, out_size)
        self.Relu = nn.ReLU()

    def forward(self, x):
        output = self.layer1(self.Relu(x))
        output = self.layer2(self.Relu(output))
        output = self.layer3(self.Relu(output))
        output = self.layer4(self.Relu(output))

        return output

## Batching

We want to separate the dataset into batches because we are using a mini-batch approach. This approach will update W's and B's at each batch's end. So we use this to update W's and B's rapidly.

In [29]:
class ChatBotDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.n_samples = len(X_train)

    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx]

    def __len__(self):
        return self.n_samples


def data_loader(X_train, y_train, batch_size):
    dataset = ChatBotDataset(X_train, y_train)

    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_loader

## Setting Hyper Parameters, Loss Functions, and Optimizer

In [30]:
inp_size = X_train.shape[1]
output_size = len(tags)
model = ChatBot(inp_size, output_size)
batch_size = 10
alpha = 0.0001
optimizer = torch.optim.Adam(model.parameters(), alpha)
loss_func = nn.CrossEntropyLoss()

## Train the model

In [31]:
def train(model, dataloader, loss_fn, optimizer, epochs):

    for epoch in range(epochs):
        for X, y in dataloader:

            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 50 == 0:
            print(f"epoch: {epoch+1}/{epochs}   loss: {loss.item():.4f}")

train_loader = data_loader(X_train, y_train, batch_size)
train(model=model, dataloader=train_loader, optimizer=optimizer, loss_fn=loss_func, epochs=320)

epoch: 50/320   loss: 2.4017
epoch: 100/320   loss: 0.3113
epoch: 150/320   loss: 0.1585
epoch: 200/320   loss: 0.0630
epoch: 250/320   loss: 0.0272
epoch: 300/320   loss: 0.0045


You can see that the loss is reduced after each epoch ends.



**NOTE: Normally we use the TEST set to evaluate the final performance of the model, but in this case, due to the small data set, I did not.**

# Design User Interface

**Note: If you're using Colab Notebook, this part of the project(user interface) will not work for you. Because Colab doesn't support the Tkinter library. So I recommend you download and run it in your code editor(like VS Code). There is file named main.py that you can download**

In [ ]:
def exit_prog(event=None):
    window.destroy()

def send(event=None):
    user_input = user_text.get()
    user_text.set('')
    if user_input == 'تمام':
        window.quit()

    chat_log.config(state=tk.NORMAL)
    chat_log.insert(tk.END, "You: " + user_input + '\n')

    tokenized = hazm.word_tokenize(user_input)
    embed = encoding(tokenized, words)
    embed = embed.reshape(1, len(embed))
    X = torch.from_numpy(embed).to(dtype=torch.float)

    output = model(X)
    _, y_pred = torch.max(output, dim=1)
    find_tag = tags[y_pred.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][y_pred.item()]

    response = ""
    if prob > 0.55:
        for instance in data["data"]:
            if instance["tag"] == find_tag:
                response = random.choice(instance["responses"])
    else:
        response = "منظورتان را درک نکردم لطفا به شیوه ی دیگری بیان کنید"

    chat_log.insert(tk.END, "Robin: " + response + '\n\n')
    chat_log.config(state=tk.DISABLED)
    chat_log.yview(tk.END)

window = tk.Tk()
window.title("Robin Chatbot: Final Bachelor Project, ALIREZA HADIPOOR")
window.configure(background='light blue')

image = Image.open("image.png")
bg_image = ImageTk.PhotoImage(image)
background_label = tk.Label(window, image=bg_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

user_text = tk.StringVar()
user_entry = tk.Entry(window, textvariable=user_text)
user_entry.pack(padx=20, pady=20)

chat_log = scrolledtext.ScrolledText(window, state=tk.DISABLED, wrap=tk.WORD, bg='light blue')
chat_log.pack(pady=10)

button_frame = tk.Frame(window, bg='gray')
button_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=55)

send_button = tk.Button(button_frame, text="Send", command=send, height=2, width=15, bg="light blue", fg='black')
send_button.pack(side=tk.LEFT, padx=250, pady=5)

exit_button = tk.Button(button_frame, text='Exit', command=exit_prog, height=2, width=15, bg="light blue", fg='black')
exit_button.pack(side=tk.RIGHT, padx=250, pady=5)

window.state('zoomed')
window.bind('<Return>', send)
window.bind('<Escape>', exit_prog)
window.mainloop()